# Import Libraries

In [1]:
from swarmintelligence import MemoryBasedGreyWolfOptimization, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools
import math
import random as rd

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 1))

In [3]:
gray_misc_dataset

{'AirplaneF16.tiff': array([[ 65, 199, 193, ..., 166, 164, 148],
        [ 54, 196, 192, ..., 179, 155, 140],
        [ 54, 197, 194, ..., 190, 159, 141],
        ...,
        [ 67, 213, 213, ..., 184, 176, 151],
        [ 68, 214, 214, ..., 166, 184, 167],
        [ 89,  86,  88, ..., 112, 111, 107]], dtype=uint8),
 'Lena.png': array([[162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        ...,
        [ 43,  43,  50, ..., 104, 100,  98],
        [ 44,  44,  55, ..., 104, 105, 108],
        [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8),
 'Male.tiff': array([[130, 138, 140, ..., 115, 113, 111],
        [139, 134, 131, ..., 110, 117, 116],
        [141, 127, 131, ..., 109, 108, 107],
        ...,
        [102, 103, 103, ...,  76,  80,  89],
        [106, 102, 109, ...,  72,  69,  82],
        [ 98, 105, 116, ...,  85,  81,  89]], dtype=uint8),
 'Mandrill.tiff': array([[145,  56,  49, ..., 113, 153, 177],

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/memGWO/optimal_params.pkl', 'rb'))
optimal_params

{'wolfSize': 25, 'maxIteration': 100, 'crossover_rate': 1.0}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['kapur_entropy']
objs = ['max']
n_runs = 30

In [6]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds':[],
    'Mean CPU_time (seconds)': [],
    'Mean Fitness':[],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions': [],
    'CPU_time (seconds)': [],
    'Fitness':[],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in gray_misc_dataset.items():
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                fitness_values = []
                list_best_thresholds = []
                list_fitness = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo = MemoryBasedGreyWolfOptimization(
                        k=threshold,
                        wolfSize=optimal_params['wolfSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        crossover_rate=optimal_params['crossover_rate'],
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs, best_thresholds = gwo.fit_run(image_array)
                    et = time.time()
                    regions = Utilization().digitize(image_array, best_thresholds)
                    list_cpu_time.append(et-st)
                    list_best_thresholds.append(best_thresholds)
                    fitness_values.append(gwo.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, regions))
                    list_rmse.append(rmse(image_array, regions))
                    list_psnr.append(psnr(image_array, regions))
                    list_ssim.append(ssim(image_array, regions)[0])
                    list_uqi.append(uqi(image_array, regions))
                    print(gwo.get_params_training_()['best_fitness_tracking'][-1])
                
                mean_best_thresholds = np.round(np.array(list_best_thresholds).mean(axis=0)).astype('int')
                mean_regions=Utilization().digitize(image_array, mean_best_thresholds)
                gwo_evaluation_results['Mean best_thresholds'].append(mean_best_thresholds)
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean Fitness'].append(np.mean(np.array(fitness_values)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions'].append(mean_regions)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['Fitness'].append(fitness_values)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean(np.array(fitness_values))}")

	 AirplaneF16.tiff | Threshold =  2
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
Mean Fitness = 12.211474090566007
	 AirplaneF16.tiff | Threshold =  3
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
1

21.53602786724339
21.57759962756542
21.335737869050963
21.56606723969977
21.55532800177896
Mean Fitness = 21.546439025193088
	 Mandrill.tiff | Threshold =  2
12.217754102632629
12.217754102632629
12.217727299731392
12.217701316049435
12.217727299731392
12.217754102632629
12.217754102632629
12.217701316049435
12.217727299731392
12.217727299731392
12.217701316049435
12.217754102632629
12.217754102632629
12.217701316049435
12.217727299731392
12.217727299731392
12.217727299731392
12.217332215039384
12.217701316049435
12.217754102632629
12.217727299731392
12.217727299731392
12.217754102632629
12.217754102632629
12.217727299731392
12.217754102632629
12.217727299731392
12.217727299731392
12.217701316049435
12.217727299731392
Mean Fitness = 12.21771786780568
	 Mandrill.tiff | Threshold =  3
15.279196147154849
15.279196147154849
15.279196147154849
15.279196147154849
15.279196147154849
15.279196147154849
15.279196147154849
15.279196147154849
15.279196147154849
15.279196147154849
15.2791961471548

21.025601459050485
21.02572017014427
21.020511382495446
20.962747907418116
20.925922027627557
21.025601459050485
21.0242408799802
20.91601612149963
21.024657837128498
20.92582640659677
20.98446793158411
21.01984646167717
21.02572017014427
Mean Fitness = 20.993633890735204


## Visualize Results using DataFrame

In [7]:
gwo_evaluation_results_df =pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds,Mean CPU_time (seconds),Mean Fitness,Mean MSE,Mean RMSE,Mean PSNR,Mean SSIM,Mean UQI,Regions,CPU_time (seconds),Fitness,MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,kapur_entropy,max,"[71, 173]",2.425600,12.211474,2806.376312,52.975242,13.649345,0.761347,0.921776,"[[71, 255, 255, 255, 255, 255, 255, 255, 255, ...","[1.927145004272461, 3.1520016193389893, 1.9857...","[12.211474090566005, 12.211474090566005, 12.21...","[2806.3763122558594, 2806.3763122558594, 2806....","[52.97524244640943, 52.97524244640943, 52.9752...","[13.64934454910294, 13.64934454910294, 13.6493...","[0.761346872735969, 0.761346872735969, 0.76134...","[0.9217762274412886, 0.9217762274412886, 0.921..."
1,AirplaneF16.tiff,3,kapur_entropy,max,"[69, 127, 183]",2.212908,15.503894,1994.712143,44.662200,15.132001,0.798145,0.949481,"[[69, 255, 255, 255, 183, 255, 255, 255, 255, ...","[2.405129909515381, 2.6317431926727295, 2.7688...","[15.503893789532656, 15.503893789532656, 15.50...","[1994.712142944336, 1994.712142944336, 1994.71...","[44.66220038180313, 44.66220038180313, 44.6622...","[15.132001293919405, 15.132001293919405, 15.13...","[0.7981453760593136, 0.7981453760593136, 0.798...","[0.949480595306752, 0.949480595306752, 0.94948..."
2,AirplaneF16.tiff,4,kapur_entropy,max,"[66, 106, 145, 185]",1.816724,18.311313,1822.272596,42.687973,15.524716,0.812972,0.956896,"[[66, 255, 255, 255, 185, 255, 255, 255, 255, ...","[2.17159366607666, 1.9110729694366455, 2.09562...","[18.31182479020967, 18.311377936178886, 18.311...","[1821.3631782531738, 1823.1306648254395, 1821....","[42.67743172044417, 42.6981342077782, 42.67906...","[15.526838084914363, 15.522625649616597, 15.52...","[0.8128127620893262, 0.8125080958432692, 0.812...","[0.9568226745745343, 0.9567879526111035, 0.956..."
3,AirplaneF16.tiff,5,kapur_entropy,max,"[60, 89, 123, 156, 187]",1.673859,20.907125,1714.852283,41.410185,15.788784,0.814105,0.961199,"[[89, 255, 255, 187, 187, 187, 255, 255, 255, ...","[1.5158076286315918, 1.6827898025512695, 1.850...","[20.9081928811917, 20.90455827590819, 20.90832...","[1722.7836265563965, 1697.8788414001465, 1723....","[41.50642873768347, 41.20532540097392, 41.5170...","[15.768496253197196, 15.831736645881884, 15.76...","[0.815175644293204, 0.8115868195759958, 0.8148...","[0.9610780784884551, 0.9615658144316607, 0.960..."
4,Lena.png,2,kapur_entropy,max,"[97, 164]",1.689077,12.346365,2044.689766,45.218246,15.024529,0.630764,0.891673,"[[164, 164, 164, 164, 164, 164, 164, 164, 255,...","[1.6800310611724854, 1.9742097854614258, 1.834...","[12.346365101175513, 12.346365101175513, 12.34...","[2044.6897659301758, 2044.6897659301758, 2044....","[45.21824594044063, 45.21824594044063, 45.2182...","[15.02452937602985, 15.02452937602985, 15.0245...","[0.6307635952929036, 0.6307635952929036, 0.630...","[0.8916733985521984, 0.8916733985521984, 0.891..."
5,Lena.png,3,kapur_entropy,max,"[82, 126, 175]",2.315866,15.318297,1204.782304,34.709949,17.321730,0.704799,0.932702,"[[175, 175, 175, 175, 175, 175, 175, 175, 175,...","[1.6595416069030762, 1.6158673763275146, 1.557...","[15.317890413903815, 15.318363633025005, 15.31...","[1200.7408065795898, 1205.9774589538574, 1205....","[34.651707123597674, 34.72718616522015, 34.727...","[17.336310907012276, 17.31741170429817, 17.317...","[0.7034553771538843, 0.7049037330390853, 0.704...","[0.9320803604502947, 0.9327013076974919, 0.932..."
6,Lena.png,4,kapur_entropy,max,"[68, 103, 142, 181]",5.102331,18.004744,821.881530,28.656029,18.990290,0.749043,0.957655,"[[181, 181, 181, 181, 181, 181, 181, 181, 181,...","[4.306861639022827, 4.9597508907318115, 5.2523...","[18.01225129920148, 18.01225129920148, 17.9995...","[865.4517211914062, 865.4517211914062, 827.339...","[29.41856082801139, 29.41856082801139, 28.7635...","[18.758375148898626, 18.758375148898626, 18.95...","[0.7460478825599545, 0.7460478825599545, 0.743...","[0.9588728637050936, 0.9588728637050936, 0.958..."
7,Lena.png,5,kapur_en

# Save results

In [8]:
pickle.dump(gwo_evaluation_results_df, open('results/evaluation/memGWO/memGWO_kapur_df.pkl', 'wb'))
pickle.dump(gwo_evaluation_results, open('results/evaluation/memGWO/memGWO_kapur_dict.pkl', 'wb'))